<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import os
from tqdm import tqdm
import tifffile as tiff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [3]:
# From https://www.kaggle.com/stainsby/fast-tested-rle
def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

In [8]:
shape = [520, 704]

dataset_folder = "dataset"
output_folder = f"{dataset_folder}/train_converted"
os.makedirs(output_folder, exist_ok=True)

df = pd.read_csv(f"{dataset_folder}/train.csv")
df = df.groupby('id').agg(list).reset_index()
for idx, row in tqdm(df.iterrows()):
    img = row['id']
    ann = row['annotation']
    image = cv2.imread(f"{dataset_folder}/train/{img}" + '.png')
    out_mask = np.zeros(shape)
    for i, mask in enumerate(ann):
        out_mask += rle_decode(mask, shape).astype(np.uint16) * (i + 1)
    # plt.imshow(out_mask)
    # plt.show()
    tiff.imsave(f"{output_folder}/{img}_masks.tif", out_mask)
    tiff.imsave(f"{output_folder}/{img}.tif", image[:, :, 0]) #single ch

606it [00:29, 20.72it/s]
